# FUME-FastSCNN Training Notebook

This notebook trains the FUME-FastSCNN model for dual-gas acidosis detection.

**Model:** FUME-FastSCNN (~2.8M parameters)
**Task:** Multi-task (Segmentation + Classification)
**Dataset:** Augmented gas emission dataset (8,967 samples)

## 1. Setup

In [1]:
import sys
sys.path.append('..')  # Add parent directory to path

import torch
import yaml
from pathlib import Path

# Check CUDA
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.8.0+cu128
CUDA available: True
CUDA device: NVIDIA RTX 6000 Ada Generation
CUDA version: 12.8


## 2. Configuration

In [2]:
# Load config
config_path = '../configs/fume_fastscnn_config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("📋 Configuration loaded:")
print(f"  Experiment: {config['experiment']['name']}")
print(f"  Model: {config['model']['name']}")
print(f"  Batch size: {config['training']['batch_size']}")
print(f"  Epochs: {config['training']['num_epochs']}")
print(f"  Learning rate: {config['training']['optimizer']['lr']}")

📋 Configuration loaded:
  Experiment: FUME-FastSCNN
  Model: FUMEFastSCNN
  Batch size: 8
  Epochs: 50
  Learning rate: 0.001


## 3. Data Preparation

**First, run the data pairing script if not already done:**

```bash
cd ../data
python pairing.py
```

In [3]:
# Check if paired annotations exist
import os

required_files = [
    '../data/paired_train_annotations.csv',
    '../data/paired_val_annotations.csv',
    '../data/paired_test_annotations.csv'
]

all_exist = all(os.path.exists(f) for f in required_files)

if all_exist:
    print("✅ All paired annotation files found!")
else:
    print("❌ Paired annotation files not found. Run data/pairing.py first!")
    print("\nRun this command in terminal:")
    print("  cd data && python pairing.py")

✅ All paired annotation files found!


## 4. Model Overview

In [4]:
from models import FUMEFastSCNN

# Create model
model = FUMEFastSCNN(
    num_classes=3,
    num_seg_classes=3,
    shared_encoder=True
)

# Count parameters
num_params = model.get_num_parameters()
print(f"\nModel Statistics:")
print(f"  Total parameters: {num_params:,}")
print(f"  Parameters (M): {num_params/1e6:.2f}M")
print(f"  Within budget: {'Yes' if num_params < 3e6 else 'No'}")

# Test forward pass (use eval mode for inference testing)
model.eval()
dummy_co2 = torch.randn(1, 1, 480, 640)
dummy_ch4 = torch.randn(1, 1, 480, 640)
dummy_mask = torch.ones(1, 2)

with torch.no_grad():
    outputs = model(dummy_co2, dummy_ch4, dummy_mask)
print(f"\nOutput shapes:")
print(f"  Classification: {outputs['cls_logits'].shape}")
print(f"  CO2 Segmentation: {outputs['co2_seg_logits'].shape}")
print(f"  CH4 Segmentation: {outputs['ch4_seg_logits'].shape}")


Model Statistics:
  Total parameters: 1,656,236
  Parameters (M): 1.66M
  Within budget: Yes

Output shapes:
  Classification: torch.Size([1, 3])
  CO2 Segmentation: torch.Size([1, 3, 480, 640])
  CH4 Segmentation: torch.Size([1, 3, 480, 640])


## 5. Start Training

**Option B: Train in notebook (for debugging)**

In [ ]:
# Import trainer
import os
sys.path.append('..')

# Change to project root (required for config relative paths)
os.chdir('..')
print(f"Working directory: {os.getcwd()}")

from train import Trainer

# Create trainer
trainer = Trainer(config_path='configs/fume_fastscnn_config.yaml')

# Start training
trainer.train()

Working directory: /home/siu856569517/Taminul/Acidosis/FUME


/home/siu856569517/.conda/envs/fume_env/lib/python3.9/site-packages/wandb/sdk/launch/builder/build.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


✅ Seed set to 42
🚀 Using device: cuda
✅ Directories created


/home/siu856569517/.conda/envs/fume_env/lib/python3.9/site-packages/wandb/sdk/launch/builder/build.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
wandb: Currently logged in as: taminul. Use `wandb login --relogin` to force relogin


INFO:data.dataset:Loaded 4383 paired samples
INFO:data.dataset:  Fully paired: 1893
INFO:data.dataset:  Modality dropout: 0.2
INFO:data.dataset:  Class distribution: {'Acidotic': 2734, 'Healthy': 1488, 'Transitional': 161}
INFO:data.dataset:Loaded 939 paired samples
INFO:data.dataset:  Fully paired: 406
INFO:data.dataset:  Modality dropout: 0.0
INFO:data.dataset:  Class distribution: {'Acidotic': 586, 'Healthy': 318, 'Transitional': 35}


...

Epoch 4/50:   0%|          | 0/548 [00:00<?, ?it/s]wandb: WARNING (User provided step: 3 is less than current step: 2144. Dropping entry: {'val_loss': 11.744354741864058, 'val_pixel_accuracy': 0.910315772541711, 'val_mean_iou': 0.7192431952749185, 'val_mean_dice': 0.8308861096623593, 'val_iou_background': 0.8947582063548074, 'val_iou_tube': 0.6384734227656287, 'val_iou_gas': 0.6244979567043195, 'val_dice_background': 0.9444563463072896, 'val_dice_tube': 0.7793515767719078, 'val_dice_gas': 0.7688504059078808, 'val_accuracy': 0.4579339723109691, 'val_balanced_accuracy': 0.3970420932878271, 'val_macro_f1': 0.2921251402426563, 'val_weighted_f1': 0.3883845734297337, 'val_cohens_kappa': 0.13587342683008385, 'val_Healthy_precision': 0.3845223700120919, 'val_Healthy_recall': 1.0, 'val_Healthy_f1': 0.5554585152838428, 'val_Transitional_precision': 0.0, 'val_Transitional_recall': 0.0, 'val_Transitional_f1': 0.0, 'val_Acidotic_precision': 1.0, 'val_Acidotic_recall': 0.19112627986348124, 'val_Acid

Epoch 4 completed in 5.18 minutes
Best metric: 0.4016
----------------------------------------------------------------------


Epoch 5/50:   0%|          | 0/548 [00:00<?, ?it/s]/home/siu856569517/Taminul/Acidosis/FUME/train.py:268: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config['training']['use_amp']):
Validating: 100%|██████████| 939/939 [00:12<00:00, 76.62it/s]



  EPOCH 5 RESULTS

  LOSS
----------------------------------------
  Train Loss                0.4830
  Val Loss                  12.5227

  SEGMENTATION METRICS
----------------------------------------
  mean_iou                  0.7352
  mean_dice                 0.8421
  pixel_accuracy            0.9180

  CLASSIFICATION METRICS
----------------------------------------
  accuracy                  0.4633
  balanced_accuracy         0.3984
  macro_f1                  0.2980
  weighted_f1               0.3994
  cohens_kappa              0.1398

  PER-CLASS F1 SCORES
----------------------------------------
  Healthy                   0.5556
  Transitional              0.0000
  Acidotic                  0.3385

Epoch 5 completed in 5.20 minutes
Best metric: 0.4016
----------------------------------------------------------------------


Epoch 6/50:   0%|          | 0/548 [00:00<?, ?it/s]/home/siu856569517/Taminul/Acidosis/FUME/train.py:268: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config['training']['use_amp']):
Epoch 6/50:  82%|████████▏ | 451/548 [04:16<00:42,  2.28it/s, loss=0.351]

## 6. Monitor Training

**Weights & Biases Dashboard:**
- View training curves in real-time
- Compare experiments
- Track system metrics

**TensorBoard (if W&B not available):**
```bash
tensorboard --logdir=logs
```

## 7. Load and Test Checkpoint

In [ ]:
# Load best checkpoint
checkpoint_path = '../checkpoints/best_model.pth'

if Path(checkpoint_path).exists():
    checkpoint = torch.load(checkpoint_path)

    # Load model
    model = FUMEFastSCNN(
        num_classes=3,
        num_seg_classes=3,
        shared_encoder=True
    )
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    print(f"✅ Loaded checkpoint from epoch {checkpoint['epoch']}")
    print(f"   Best metric: {checkpoint['best_metric']:.4f}")
else:
    print("❌ No checkpoint found. Train the model first!")

## 8. Training Summary

After training completes, check:
- `checkpoints/best_model.pth` - Best model weights
- `checkpoints/last_model.pth` - Latest model weights
- `logs/` - Training logs
- W&B dashboard - Training curves and metrics

## Next Steps

1. ✅ Training completed
2. 📊 Evaluate on test set (use `test_fume.ipynb`)
3. 📈 Compare with baselines
4. 🔬 Run ablation studies
5. 📝 Generate paper figures